In [40]:
# DB instance identifier : investdata
# username: name : admin
# Password: 12345678
# Host: investdata.c5cwsai4kiot.us-east-1.rds.amazonaws.com
# Port: 3306
# Region = us-east-1a

In [3]:
!pip3 install pandas pymysql


1373.58s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ee/0d/43c80cc69ad223356b61d12e488acf5f861dc833d498f43869efd871f5c9/pandas-2.2.0-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/a3/fb/52b62131e21b24ee297e4e95ed41eba29647dad0e0051a92bb66b43c70ff/tzdata-2023.4-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 9.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 12.2 MB/s eta 0:00:00

[notice] A new release of pip is 

In [2]:
import pymysql
db = pymysql.connect(host = 'investdata.c5cwsai4kiot.us-east-1.rds.amazonaws.com',user = 'admin',password ='12345678')

cursor = db.cursor()
cursor.execute("select version()")

1

In [61]:
import pandas as pd
import pymysql
import os
import re
import glob

# Database connection parameters
host = os.environ['DB_HOST']
user = os.environ['DB_USER']
password = os.environ['DB_PASSWORD']
database = os.environ['DB_NAME']



# Directory containing CSV files
directory_path = '/Users/han-a/Desktop/Stockdata'

# Connect to the database
db = pymysql.connect(host=host, user=user, password=password, database=database)
cursor = db.cursor()

# Create table SQL statement
create_table_sql = """
CREATE TABLE IF NOT EXISTS stock_data (
    timestamp DATETIME,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume BIGINT,
    symbol VARCHAR(10)
)
"""

# Execute the create table statement
cursor.execute(create_table_sql)

# Function to upload a single file using bulk insert
def upload_file(csv_file_path):
    filename = os.path.basename(csv_file_path)
    symbol_match = re.search(r'\w+_\w+_(\w+)_\d{4}-\d{2}\.csv$', filename)
    if symbol_match:
        symbol = symbol_match.group(1).upper()
    else:
        print(f"Could not extract symbol from filename {filename}")
        return  # Skip this file if the symbol cannot be extracted
    
    df = pd.read_csv(csv_file_path)
    df['symbol'] = symbol

    # Prepare data for bulk insert
    data_tuples = [tuple(x) for x in df.to_records(index=False)]
    
    insert_sql = """
    INSERT INTO stock_data (timestamp, open, high, low, close, volume, symbol)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """

    # Execute bulk insert
    try:
        cursor.executemany(insert_sql, data_tuples)
        db.commit()
        print(f"Data from {csv_file_path} uploaded successfully.")
    except Exception as e:
        db.rollback()  # Rollback in case of error
        print(f"Failed to upload data from {csv_file_path}. Error: {e}")

# Loop through each CSV file in the directory and upload it
for csv_file_path in glob.glob(os.path.join(directory_path, '*.csv')):
    upload_file(csv_file_path)

# Close the database connection
db.close()


Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2020-09.csv uploaded successfully.
Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2020-08.csv uploaded successfully.
Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2021-07.csv uploaded successfully.
Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2021-06.csv uploaded successfully.
Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2021-12.csv uploaded successfully.
Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2021-04.csv uploaded successfully.
Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2021-10.csv uploaded successfully.
Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2021-11.csv uploaded successfully.
Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2021-05.csv uploaded successfully.
Data from /Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2021-01.csv uploaded successfully.
Data from /Users/han

In [ ]:
import os

host = os.environ['DB_HOST']
user = os.environ['DB_USER']
password = os.environ['DB_PASSWORD']
database = os.environ['DB_NAME']


try:
    # Connect to the MySQL server
    connection = pymysql.connect(host=host, user=user, password=password, database=database)

    # Create a cursor object
    cursor = connection.cursor()

    # Define the SQL query to select all data from the table
    select_query = f"SELECT * FROM {'stock_data'}"

    # Execute the query
    cursor.execute(select_query)

    # Fetch all the rows (table data)
    table_data = cursor.fetchall()

    # Display the table data
    for row in table_data:
        print(row)

except pymysql.Error as e:
    print(f"Error: {e}")

finally:
    if connection:
        connection.close()

In [47]:

cursor.fetchall()

[]

In [50]:
# db.close()
cursor.close()

In [54]:
# cursor.close()

# Close the database connection
db.close()

Error: Already closed

In [35]:
import re
import os

csv_file_path = '/Users/han-a/Desktop/Stockdata/alphavantage_data_GME_2020-01.csv'
filename = os.path.basename(csv_file_path)
string = filename
match = re.search(r'\w+_\w+_(\w+)_\d{4}-\d{2}\.csv$', filename)



if match:
    symbol = match.group(1)
    print("Symbol:", symbol)
else:
    print("No match found.")


# CSV file path


# # Extract symbol from filename (assumed format: x-x-symbol-x.csv)
# filename = os.path.basename(csv_file_path)
# symbol_match = re.search(r'-([^-]+)-[^-]+\.csv$', filename)


rewq
Symbol: GME
